The following tutorial is available on the [Wallaroo Github Repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2025.1_tutorials/wallaroo-run-anywhere/inference-on-any-architecture/x86/edge-classification-finserv).

## Classification Financial Services Edge Deployment Tutorial

This notebook will walk through building Wallaroo pipeline with a a Classification model deployed to detect the likelihood of credit card fraud, then publishing that pipeline to an Open Container Initiative (OCI) Registry where it can be deployed in other Docker and Kubernetes environments.  This example uses the Wallaroo SDK.

This demonstration will focus on deployment to the edge.

This demonstration performs the following:

1. As a Data Scientist in Wallaroo Ops:
    1. Upload a computer vision model to Wallaroo, deploy it in a Wallaroo pipeline, then perform a sample inference.
    2. Publish the pipeline to an Open Container Initiative (OCI) Registry service.  This is configured in the Wallaroo instance.  See [Edge Deployment Registry Guide](https://docs.wallaroo.ai/wallaroo-platform-operations/wallaroo-platform-operations-configure/wallaroo-edge-deployment/) for details on adding an OCI Registry Service to Wallaroo as the Edge Deployment Registry.  This demonstration uses a GitHub repository - see [Introduction to GitHub Packages](https://docs.github.com/en/packages/learn-github-packages/introduction-to-github-packages) for setting up your own package repository using GitHub, which can then be used with this tutorial.
    3. View the pipeline publish details.
2. As a DevOps Engineer in a remote aka edge device:
    1. Deploy the published pipeline as a Wallaroo Inference Server.  This example will use Docker.
    2. Perform a sample inference through the Wallaroo Inference Server with the same data used in the data scientist example.

## References

* [Wallaroo SDK Essentials Guide: Pipeline Edge Publication](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-publication/)

## Data Scientist Pipeline Publish Steps

### Load Libraries

The first step is to import the libraries used in this notebook.

In [2]:
import wallaroo
from wallaroo.object import EntityNotFoundError

import pyarrow as pa
import pandas as pd
import requests


# used to display dataframe information without truncating
from IPython.display import display
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

### Connect to the Wallaroo Instance through the User Interface

The next step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [3]:
wl = wallaroo.Client()

### Create a New Workspace

We'll use the SDK below to create our workspace , assign as our **current workspace**, then display all of the workspaces we have at the moment.  We'll also set up variables for our models and pipelines down the road, so we have one spot to change names to whatever fits your organization's standards best.

To allow this tutorial to be run by multiple users in the same Wallaroo instance, a random 4 character prefix will be added to the workspace, pipeline, and model.  Feel free to set `suffix=''` if this is not required.

In [4]:
workspace_name = f'edge-publish-demo'
pipeline_name = 'edge-pipeline'
xgboost_model_name = 'ccfraud-xgboost'
xgboost_model_file_name = './models/xgboost_ccfraud.onnx'
keras_model_name = 'ccfraud-keras'
keras_model_file_name = './models/keras_ccfraud.onnx'

In [5]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)

wl.set_current_workspace(workspace)

{'name': 'edge-publish-demo', 'id': 1549, 'archived': False, 'created_by': 'jonathan@wallaroo.ai', 'created_at': '2025-05-07T17:37:12.775631+00:00', 'models': [{'name': 'ccfraud-xgboost', 'versions': 10, 'owner_id': '""', 'last_update_time': datetime.datetime(2025, 5, 13, 19, 18, 6, 266277, tzinfo=tzutc()), 'created_at': datetime.datetime(2025, 5, 7, 17, 37, 14, 28703, tzinfo=tzutc())}, {'name': 'ccfraud-keras', 'versions': 10, 'owner_id': '""', 'last_update_time': datetime.datetime(2025, 5, 13, 19, 18, 6, 793447, tzinfo=tzutc()), 'created_at': datetime.datetime(2025, 5, 7, 17, 37, 14, 505459, tzinfo=tzutc())}], 'pipelines': [{'name': 'edge-pipeline', 'create_time': datetime.datetime(2025, 5, 7, 17, 37, 19, 80726, tzinfo=tzutc()), 'definition': '[]'}]}

In [8]:
pub = wl.list_publishes(1549)[0]
pub

ID,34
Pipeline Name,edge-pipeline
Pipeline Version,4127d1f7-825c-4226-b012-bf8a06104a8f
Status,Published
Workspace Id,1549
Workspace Name,edge-publish-demo
Edges,
Engine URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-main-6131
Pipeline URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-pipeline:4127d1f7-825c-4226-b012-bf8a06104a8f
Helm Chart URL,oci://us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts/edge-pipeline
Helm Chart Reference,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts@sha256:0b6e292038cb208f725fcee4b52a8ed7c4a7a99f63397f28f949762eca0dba28


In [10]:
edge_name = 'cv-observability-demo-sample'
edge_publish = pub.add_edge(edge_name)
display(edge_publish)

ID,34
Pipeline Name,edge-pipeline
Pipeline Version,4127d1f7-825c-4226-b012-bf8a06104a8f
Status,Published
Workspace Id,1549
Workspace Name,edge-publish-demo
Edges,cv-observability-demo-sample
Engine URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-main-6131
Pipeline URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-pipeline:4127d1f7-825c-4226-b012-bf8a06104a8f
Helm Chart URL,oci://us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts/edge-pipeline
Helm Chart Reference,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts@sha256:0b6e292038cb208f725fcee4b52a8ed7c4a7a99f63397f28f949762eca0dba28


### Upload the Model

When a model is uploaded to a Wallaroo cluster, it is optimized and packaged to make it ready to run as part of a pipeline. In many times, the Wallaroo Server can natively run a model without any Python overhead. In other cases, such as a Python script, a custom Python environment will be automatically generated. This is comparable to the process of "containerizing" a model by adding a small HTTP server and other wrapping around it.

Our pretrained model is in ONNX format, which is specified in the `framework` parameter.

In [5]:
xgboost_edge_demo_model = wl.upload_model(
    xgboost_model_name,
    xgboost_model_file_name,
    framework=wallaroo.framework.Framework.ONNX,
).configure(tensor_fields=["tensor"])

keras_edge_demo_model = wl.upload_model(
    keras_model_name,
    keras_model_file_name,
    framework=wallaroo.framework.Framework.ONNX,
).configure(tensor_fields=["tensor"])

### Reserve Pipeline Resources

Before deploying an inference engine we need to tell wallaroo what resources it will need.
To do this we will use the wallaroo DeploymentConfigBuilder() and fill in the options listed below to determine what the properties of our inference engine will be.

We will be testing this deployment for an edge scenario, so the resource specifications are kept small -- what's the minimum needed to meet the expected load on the planned hardware.

- cpus - 1 => allow the engine to use 4 CPU cores when running the neural net
- memory - 900Mi => each inference engine will have 2 GB of memory, which is plenty for processing a single image at a time.

In [6]:
deploy_config = (wallaroo
                 .DeploymentConfigBuilder()
                 .replica_count(1)
                 .cpus(1)
                 .memory("900Mi")
                 .build()
                 )

### Simulated Edge Deployment

We will now deploy our pipeline into the current Kubernetes environment using the specified resource constraints. This is a "simulated edge" deploy in that we try to mimic the edge hardware as closely as possible.

In [7]:
pipeline = wl.build_pipeline(pipeline_name)

# clear the pipeline if previously run
pipeline.clear()
pipeline.add_model_step(xgboost_edge_demo_model)

pipeline.deploy(deployment_config = deploy_config, wait_for_status=False)


Deployment initiated for edge-pipeline. Please check pipeline status.


name,edge-pipeline
created,2025-05-07 17:37:19.080726+00:00
last_updated,2025-05-13 19:18:07.212658+00:00
deployed,True
workspace_id,1549
workspace_name,edge-publish-demo
arch,x86
accel,none
tags,
versions,"3baee91b-7adb-4ee3-b04e-bb4176c2126d, 4e78e5a3-6a12-4bac-aadc-0bf5eceefccd, 59924841-88f1-4f7b-8dc6-bc0b1d6ec65c, fd9e3a36-1c75-4982-bb17-7efbf6a63543, 9e4b0ab9-e935-412c-8e2c-7028138d43db, 3851f64f-97c3-49fe-86fe-2d1b6b8474b6, c6bbb077-fecb-4694-9797-6e7e0cb5c7e8, d6859c4b-cda5-4cd8-a95b-b531af4d4741, 63548b0c-efa1-4127-9621-e941716eee9a, b61426f4-415c-45bb-86de-989823e6469a, ba314f16-4a35-4a05-bbc1-a1d29b7b8587, fa08e559-1b61-48bf-9b6b-997cd9498a41, 200a5862-0ac6-4767-b22d-8c13d339f17a, 644f6bdd-94f9-4040-873c-ca6f4243c205, 8625acbf-1ee5-48fa-92e9-7539468fb5fa, bd3e092f-59a1-45f3-a6f7-1f857572d587, 3a7af18e-5133-4a70-83ca-87dbdb330cb0, 4127d1f7-825c-4226-b012-bf8a06104a8f, 2837e82c-3b1e-4772-babd-bbaf4a5ce480, 9babd339-ae4d-4f96-a3d6-160064570432, 5ca50d6d-d89d-4e8c-8d75-a1cfb72547e7, 10dd5d3d-f3a2-4b8c-b26e-c12ae4e250ad, f0070660-582d-4f3f-9a77-750c316f254b, 3071aabd-cc4d-41b3-8ee1-2fc995db8df5, 09bb12f5-19d5-48e7-87cf-6a840bf02f51, c846ee3b-0909-4e3d-9eca-2ed806e72c6d, 034e4372-5608-4e1f-ae35-77b3e3bc2961, fda37f8b-8342-4bed-8344-39b23a49e075, cbdc71fd-7332-44c6-874f-77d4249ac766"
steps,ccfraud-xgboost


In [8]:
import time
time.sleep(15)

while pipeline.status()['status'] != 'Running':
    time.sleep(15)
    print("Waiting for deployment.")
pipeline.status()['status']

# get this pipeline version

pipeline_xgb_version = wl.get_pipeline(pipeline_name)

### Run Single Inference

A single image, encoded using the Apache Arrow format, is sent to the deployed pipeline. Arrow is used here because, as a binary protocol, there is far lower network and compute overhead than using JSON. The Wallaroo Server engine accepts both JSON, pandas DataFrame, and Apache Arrow formats.

The sample DataFrames and arrow tables are in the `./data` directory.  We'll use the Apache Arrow table `cc_data_10k.arrow`.

In [9]:
deploy_url = pipeline._deployment._url()

headers = wl.auth.auth_header()

headers['Content-Type']='application/vnd.apache.arrow.file'
# headers['Content-Type']='application/json; format=pandas-records'
headers['Accept']='application/json; format=pandas-records'

dataFile = './data/cc_data_10k.arrow'

In [10]:
!curl -X POST {deploy_url} \
     -H "Authorization:{headers['Authorization']}" \
     -H "Content-Type:{headers['Content-Type']}" \
     -H "Accept:{headers['Accept']}" \
     --data-binary @{dataFile} > curl_response_xgboost.df.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8222k  100 7059k  100 1162k  59.4M   9.7M --:--:-- --:--:-- --:--:-- 69.8M


We'll retrieve the DataFrame from our recent inference and display the first 5 rows.

In [11]:
df = pd.read_json('curl_response_xgboost.df.json', orient="records")
df.head(5)

,time,in,out,anomaly,metadata
0,1747163905729,"{'tensor': [-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.7687824400000001, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204422999999999, -0.10414918000000001, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463211999999999]}",{'variable': [1.0094898]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-xgboost"",""model_sha"":""054810e3e3ebbdd34438d9c1a08ed6a6680ef10bf97b9223f78ebf38e14b3b52""}', 'pipeline_version': '3baee91b-7adb-4ee3-b04e-bb4176c2126d', 'elapsed': [1771916, 16883862], 'dropped': [], 'partition': 'engine-7d6f4f75fc-2t5kn'}"
1,1747163905729,"{'tensor': [-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.7687824400000001, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204422999999999, -0.10414918000000001, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463211999999999]}",{'variable': [1.0094898]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-xgboost"",""model_sha"":""054810e3e3ebbdd34438d9c1a08ed6a6680ef10bf97b9223f78ebf38e14b3b52""}', 'pipeline_version': '3baee91b-7adb-4ee3-b04e-bb4176c2126d', 'elapsed': [1771916, 16883862], 'dropped': [], 'partition': 'engine-7d6f4f75fc-2t5kn'}"
2,1747163905729,"{'tensor': [-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.7687824400000001, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204422999999999, -0.10414918000000001, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463211999999999]}",{'variable': [1.0094898]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-xgboost"",""model_sha"":""054810e3e3ebbdd34438d9c1a08ed6a6680ef10bf97b9223f78ebf38e14b3b52""}', 'pipeline_version': '3baee91b-7adb-4ee3-b04e-bb4176c2126d', 'elapsed': [1771916, 16883862], 'dropped': [], 'partition': 'engine-7d6f4f75fc-2t5kn'}"
3,1747163905729,"{'tensor': [-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.7687824400000001, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204422999999999, -0.10414918000000001, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463211999999999]}",{'variable': [1.0094898]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-xgboost"",""model_sha"":""054810e3e3ebbdd34438d9c1a08ed6a6680ef10bf97b9223f78ebf38e14b3b52""}', 'pipeline_version': '3baee91b-7adb-4ee3-b04e-bb4176c2126d', 'elapsed': [1771916, 16883862], 'dropped': [], 'partition': 'engine-7d6f4f75fc-2t5kn'}"
4,1747163905729,"{'tensor': [0.5817662, 0.09788155, 0.15468194, 0.4754102, -0.19788623, -0.45043448, 0.016654044, -0.025607055, 0.09205616, -0.27839172, 0.059329946, -0.019658541, -0.42250833, -0.12175389, 1.5473095, 0.23916228, 0.3553975, -0.76851654, -0.7000849, -0.11900433, -0.34505169999999996, -1.1065114, 0.25234112000000003, 0.020944182000000002, 0.21992674, 0.25406894, -0.04502251, 0.10867739, 0.25471792]}",{'variable': [-1.9073485999999998e-06]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-xgboost"",""model_sha"":""054810e3e3ebbdd34438d9c1a08ed6a6680ef10bf97b9223f78ebf38e14b3b52""}', 'pipeline_version': '3baee91b-7adb-4ee3-b04e-bb4176c2126d', 'elapsed': [1771916, 16883862], 'dropped': [], 'partition': 'engine-7d6f4f75fc-2t5kn'}"


### Redeploy with Keras version

We will now redeploy the pipeline with the converted keras version of our model, then set that version of the pipeline.

In [12]:
# clear the pipeline steps
pipeline.undeploy()
pipeline.clear()
pipeline.add_model_step(keras_edge_demo_model)
_ = pipeline.deploy(deployment_config = deploy_config, wait_for_status=False)

Waiting for undeployment - this will take up to 45s .................................... ok
Deployment initiated for edge-pipeline. Please check pipeline status.


We will retrieve this pipeline version, then perform our sample inference request with the keras model.

In [13]:
# give a moment to verify the model swap
time.sleep(15)

while pipeline.status()['status'] != 'Running':
    time.sleep(15)
    print("Waiting for deployment.")
    pipeline.status()['status']
pipeline.status()['status']

# get this version
keras_pipeline_version = wl.get_pipeline(pipeline_name)
keras_pipeline_version

name,edge-pipeline
created,2025-05-07 17:37:19.080726+00:00
last_updated,2025-05-13 19:19:05.090149+00:00
deployed,True
workspace_id,1549
workspace_name,edge-publish-demo
arch,x86
accel,none
tags,
versions,"f739ef8c-f97f-4a11-ac2f-eacf8c2544d2, 3baee91b-7adb-4ee3-b04e-bb4176c2126d, 4e78e5a3-6a12-4bac-aadc-0bf5eceefccd, 59924841-88f1-4f7b-8dc6-bc0b1d6ec65c, fd9e3a36-1c75-4982-bb17-7efbf6a63543, 9e4b0ab9-e935-412c-8e2c-7028138d43db, 3851f64f-97c3-49fe-86fe-2d1b6b8474b6, c6bbb077-fecb-4694-9797-6e7e0cb5c7e8, d6859c4b-cda5-4cd8-a95b-b531af4d4741, 63548b0c-efa1-4127-9621-e941716eee9a, b61426f4-415c-45bb-86de-989823e6469a, ba314f16-4a35-4a05-bbc1-a1d29b7b8587, fa08e559-1b61-48bf-9b6b-997cd9498a41, 200a5862-0ac6-4767-b22d-8c13d339f17a, 644f6bdd-94f9-4040-873c-ca6f4243c205, 8625acbf-1ee5-48fa-92e9-7539468fb5fa, bd3e092f-59a1-45f3-a6f7-1f857572d587, 3a7af18e-5133-4a70-83ca-87dbdb330cb0, 4127d1f7-825c-4226-b012-bf8a06104a8f, 2837e82c-3b1e-4772-babd-bbaf4a5ce480, 9babd339-ae4d-4f96-a3d6-160064570432, 5ca50d6d-d89d-4e8c-8d75-a1cfb72547e7, 10dd5d3d-f3a2-4b8c-b26e-c12ae4e250ad, f0070660-582d-4f3f-9a77-750c316f254b, 3071aabd-cc4d-41b3-8ee1-2fc995db8df5, 09bb12f5-19d5-48e7-87cf-6a840bf02f51, c846ee3b-0909-4e3d-9eca-2ed806e72c6d, 034e4372-5608-4e1f-ae35-77b3e3bc2961, fda37f8b-8342-4bed-8344-39b23a49e075, cbdc71fd-7332-44c6-874f-77d4249ac766"
steps,ccfraud-keras


In [14]:
deploy_url = pipeline._deployment._url()

headers = wl.auth.auth_header()

headers['Content-Type']='application/vnd.apache.arrow.file'
headers['Accept']='application/json; format=pandas-records'

dataFile = './data/cc_data_10k.arrow'

In [15]:
import datetime

local_inference_start = datetime.datetime.now()

In [18]:
!curl -X POST {deploy_url} \
     -H "Authorization:{headers['Authorization']}" \
     -H "Content-Type:{headers['Content-Type']}" \
     -H "Accept:{headers['Accept']}" \
     --data-binary @{dataFile} > curl_response_keras.df.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8161k  100 6999k  100 1162k  75.2M  12.5M --:--:-- --:--:-- --:--:-- 88.5M


We'll retrieve the DataFrame from our recent inference and display the first 5 rows.

In [19]:
df = pd.read_json('curl_response_keras.df.json', orient="records")
df.head(5)

,time,in,out,anomaly,metadata
0,1747164017683,"{'tensor': [-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.7687824400000001, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204422999999999, -0.10414918000000001, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463211999999999]}",{'dense_1': [0.99300325]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-keras"",""model_sha"":""bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507""}', 'pipeline_version': 'f739ef8c-f97f-4a11-ac2f-eacf8c2544d2', 'elapsed': [1301659, 5162166], 'dropped': [], 'partition': 'engine-c49c89849-pwx2h'}"
1,1747164017683,"{'tensor': [-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.7687824400000001, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204422999999999, -0.10414918000000001, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463211999999999]}",{'dense_1': [0.99300325]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-keras"",""model_sha"":""bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507""}', 'pipeline_version': 'f739ef8c-f97f-4a11-ac2f-eacf8c2544d2', 'elapsed': [1301659, 5162166], 'dropped': [], 'partition': 'engine-c49c89849-pwx2h'}"
2,1747164017683,"{'tensor': [-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.7687824400000001, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204422999999999, -0.10414918000000001, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463211999999999]}",{'dense_1': [0.99300325]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-keras"",""model_sha"":""bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507""}', 'pipeline_version': 'f739ef8c-f97f-4a11-ac2f-eacf8c2544d2', 'elapsed': [1301659, 5162166], 'dropped': [], 'partition': 'engine-c49c89849-pwx2h'}"
3,1747164017683,"{'tensor': [-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.7687824400000001, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204422999999999, -0.10414918000000001, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463211999999999]}",{'dense_1': [0.99300325]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-keras"",""model_sha"":""bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507""}', 'pipeline_version': 'f739ef8c-f97f-4a11-ac2f-eacf8c2544d2', 'elapsed': [1301659, 5162166], 'dropped': [], 'partition': 'engine-c49c89849-pwx2h'}"
4,1747164017683,"{'tensor': [0.5817662, 0.09788155, 0.15468194, 0.4754102, -0.19788623, -0.45043448, 0.016654044, -0.025607055, 0.09205616, -0.27839172, 0.059329946, -0.019658541, -0.42250833, -0.12175389, 1.5473095, 0.23916228, 0.3553975, -0.76851654, -0.7000849, -0.11900433, -0.34505169999999996, -1.1065114, 0.25234112000000003, 0.020944182000000002, 0.21992674, 0.25406894, -0.04502251, 0.10867739, 0.25471792]}",{'dense_1': [0.0010916889]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-keras"",""model_sha"":""bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507""}', 'pipeline_version': 'f739ef8c-f97f-4a11-ac2f-eacf8c2544d2', 'elapsed': [1301659, 5162166], 'dropped': [], 'partition': 'engine-c49c89849-pwx2h'}"


### Undeploy Pipeline

With our testing complete, we will undeploy the pipeline and return the resources back to the cluster.

In [20]:
pipeline.undeploy()

Waiting for undeployment - this will take up to 45s .................................... ok


name,edge-pipeline
created,2025-05-07 17:37:19.080726+00:00
last_updated,2025-05-13 19:19:05.090149+00:00
deployed,False
workspace_id,1549
workspace_name,edge-publish-demo
arch,x86
accel,none
tags,
versions,"f739ef8c-f97f-4a11-ac2f-eacf8c2544d2, 3baee91b-7adb-4ee3-b04e-bb4176c2126d, 4e78e5a3-6a12-4bac-aadc-0bf5eceefccd, 59924841-88f1-4f7b-8dc6-bc0b1d6ec65c, fd9e3a36-1c75-4982-bb17-7efbf6a63543, 9e4b0ab9-e935-412c-8e2c-7028138d43db, 3851f64f-97c3-49fe-86fe-2d1b6b8474b6, c6bbb077-fecb-4694-9797-6e7e0cb5c7e8, d6859c4b-cda5-4cd8-a95b-b531af4d4741, 63548b0c-efa1-4127-9621-e941716eee9a, b61426f4-415c-45bb-86de-989823e6469a, ba314f16-4a35-4a05-bbc1-a1d29b7b8587, fa08e559-1b61-48bf-9b6b-997cd9498a41, 200a5862-0ac6-4767-b22d-8c13d339f17a, 644f6bdd-94f9-4040-873c-ca6f4243c205, 8625acbf-1ee5-48fa-92e9-7539468fb5fa, bd3e092f-59a1-45f3-a6f7-1f857572d587, 3a7af18e-5133-4a70-83ca-87dbdb330cb0, 4127d1f7-825c-4226-b012-bf8a06104a8f, 2837e82c-3b1e-4772-babd-bbaf4a5ce480, 9babd339-ae4d-4f96-a3d6-160064570432, 5ca50d6d-d89d-4e8c-8d75-a1cfb72547e7, 10dd5d3d-f3a2-4b8c-b26e-c12ae4e250ad, f0070660-582d-4f3f-9a77-750c316f254b, 3071aabd-cc4d-41b3-8ee1-2fc995db8df5, 09bb12f5-19d5-48e7-87cf-6a840bf02f51, c846ee3b-0909-4e3d-9eca-2ed806e72c6d, 034e4372-5608-4e1f-ae35-77b3e3bc2961, fda37f8b-8342-4bed-8344-39b23a49e075, cbdc71fd-7332-44c6-874f-77d4249ac766"
steps,ccfraud-keras


### Publish the Pipeline for Edge Deployment

It worked! For a demo, we'll take working once as "tested". So now that we've tested our pipeline, we are ready to publish it for edge deployment.

Publishing it means assembling all of the configuration files and model assets and pushing them to an Open Container Initiative (OCI) repository set in the Wallaroo instance as the Edge Registry service.  DevOps engineers then retrieve that image and deploy it through Docker, Kubernetes, or similar deployments.

See [Edge Deployment Registry Guide](https://docs.wallaroo.ai/wallaroo-platform-operations/wallaroo-platform-operations-configure/wallaroo-edge-deployment/) for details on adding an OCI Registry Service to Wallaroo as the Edge Deployment Registry.

This is done through the SDK command `wallaroo.pipeline.publish(deployment_config)`.

### Publish Example

We will now publish the pipeline to our Edge Deployment Registry with the `pipeline.publish(deployment_config)` command.  `deployment_config` is an optional field that specifies the pipeline deployment.  This can be overridden by the DevOps engineer during deployment.

In [21]:
xgb_pub=pipeline_xgb_version.publish(deploy_config)
display(xgb_pub)
keras_pub=keras_pipeline_version.publish(deploy_config)
display(keras_pub)

Waiting for pipeline publish... It may take up to 600 sec.
... Published.blishing.


ID,42
Pipeline Name,edge-pipeline
Pipeline Version,1a8eaf43-daa6-4fd8-962a-0f6ef4bd7d58
Status,Published
Workspace Id,1549
Workspace Name,edge-publish-demo
Edges,
Engine URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-main-6139
Pipeline URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-pipeline:1a8eaf43-daa6-4fd8-962a-0f6ef4bd7d58
Helm Chart URL,oci://us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts/edge-pipeline
Helm Chart Reference,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts@sha256:79ff74b43ffa725b174ffdc0d3e57a7e620adf247e4344fe605ef9acfe7584b7


Waiting for pipeline publish... It may take up to 600 sec.
... Published.blishing.


ID,43
Pipeline Name,edge-pipeline
Pipeline Version,55139e62-536e-4b15-b3dd-b1c2a8456c57
Status,Published
Workspace Id,1549
Workspace Name,edge-publish-demo
Edges,
Engine URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-main-6139
Pipeline URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-pipeline:55139e62-536e-4b15-b3dd-b1c2a8456c57
Helm Chart URL,oci://us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts/edge-pipeline
Helm Chart Reference,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts@sha256:d9ac7eb9bb201af8c927aebc001f305b1e5051642fc53e847436e7b902aa1573


### List Published Pipelines

The method `wallaroo.client.list_pipelines()` shows a list of all pipelines in the Wallaroo instance, and includes the `published` field that indicates whether the pipeline was published to the registry (`True`), or has not yet been published (`False`).

In [22]:
wl.list_pipelines(workspace_name=workspace_name)

name,created,last_updated,deployed,workspace_id,workspace_name,arch,accel,tags,versions,steps,published
edge-pipeline,2025-07-May 17:37:19,2025-13-May 19:21:32,False,1549,edge-publish-demo,x86,none,,"55139e62-536e-4b15-b3dd-b1c2a8456c57, 1a8eaf43-daa6-4fd8-962a-0f6ef4bd7d58, f739ef8c-f97f-4a11-ac2f-eacf8c2544d2, 3baee91b-7adb-4ee3-b04e-bb4176c2126d, 4e78e5a3-6a12-4bac-aadc-0bf5eceefccd, 59924841-88f1-4f7b-8dc6-bc0b1d6ec65c, fd9e3a36-1c75-4982-bb17-7efbf6a63543, 9e4b0ab9-e935-412c-8e2c-7028138d43db, 3851f64f-97c3-49fe-86fe-2d1b6b8474b6, c6bbb077-fecb-4694-9797-6e7e0cb5c7e8, d6859c4b-cda5-4cd8-a95b-b531af4d4741, 63548b0c-efa1-4127-9621-e941716eee9a, b61426f4-415c-45bb-86de-989823e6469a, ba314f16-4a35-4a05-bbc1-a1d29b7b8587, fa08e559-1b61-48bf-9b6b-997cd9498a41, 200a5862-0ac6-4767-b22d-8c13d339f17a, 644f6bdd-94f9-4040-873c-ca6f4243c205, 8625acbf-1ee5-48fa-92e9-7539468fb5fa, bd3e092f-59a1-45f3-a6f7-1f857572d587, 3a7af18e-5133-4a70-83ca-87dbdb330cb0, 4127d1f7-825c-4226-b012-bf8a06104a8f, 2837e82c-3b1e-4772-babd-bbaf4a5ce480, 9babd339-ae4d-4f96-a3d6-160064570432, 5ca50d6d-d89d-4e8c-8d75-a1cfb72547e7, 10dd5d3d-f3a2-4b8c-b26e-c12ae4e250ad, f0070660-582d-4f3f-9a77-750c316f254b, 3071aabd-cc4d-41b3-8ee1-2fc995db8df5, 09bb12f5-19d5-48e7-87cf-6a840bf02f51, c846ee3b-0909-4e3d-9eca-2ed806e72c6d, 034e4372-5608-4e1f-ae35-77b3e3bc2961, fda37f8b-8342-4bed-8344-39b23a49e075, cbdc71fd-7332-44c6-874f-77d4249ac766",ccfraud-keras,True


### List Publishes from a Pipeline

All publishes created from a pipeline are displayed with the `wallaroo.pipeline.publishes` method.  The `pipeline_version_id` is used to know what version of the pipeline was used in that specific publish.  This allows for pipelines to be updated over time, and newer versions to be sent and tracked to the Edge Deployment Registry service.

#### List Publishes Parameters

N/A

#### List Publishes Returns

A List of the following fields:

| Field | Type | Description |
|---|---|---|
| id | integer | Numerical Wallaroo id of the published pipeline. |
| pipeline_version_id | integer | Numerical Wallaroo id of the pipeline version published. |
| engine_url | string | The URL of the published pipeline engine in the edge registry. |
| pipeline_url | string | The URL of the published pipeline in the edge registry. |
| created_by | string | The email address of the user that published the pipeline.
| Created At | DateTime | When the published pipeline was created. |
| Updated At | DateTime | When the published pipeline was updated. |

In [23]:
pipeline.publishes()

id,Pipeline Name,Pipeline Version,Workspace Id,Workspace Name,Edges,Engine URL,Pipeline URL,Created By,Created At,Updated At
42,edge-pipeline,1a8eaf43-daa6-4fd8-962a-0f6ef4bd7d58,1549,edge-publish-demo,,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-main-6139,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-pipeline:1a8eaf43-daa6-4fd8-962a-0f6ef4bd7d58,john.hummel@wallaroo.ai,2025-13-May 19:21:16,2025-13-May 19:21:16
43,edge-pipeline,55139e62-536e-4b15-b3dd-b1c2a8456c57,1549,edge-publish-demo,,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-main-6139,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-pipeline:55139e62-536e-4b15-b3dd-b1c2a8456c57,john.hummel@wallaroo.ai,2025-13-May 19:21:32,2025-13-May 19:21:32
33,edge-pipeline,5ca50d6d-d89d-4e8c-8d75-a1cfb72547e7,1549,edge-publish-demo,,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-main-6125,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-pipeline:5ca50d6d-d89d-4e8c-8d75-a1cfb72547e7,jonathan@wallaroo.ai,2025-07-May 17:58:13,2025-07-May 17:58:13
34,edge-pipeline,4127d1f7-825c-4226-b012-bf8a06104a8f,1549,edge-publish-demo,,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-main-6131,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-pipeline:4127d1f7-825c-4226-b012-bf8a06104a8f,jonathan@wallaroo.ai,2025-08-May 15:28:37,2025-08-May 15:28:37
35,edge-pipeline,200a5862-0ac6-4767-b22d-8c13d339f17a,1549,edge-publish-demo,,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-main-6131,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-pipeline:200a5862-0ac6-4767-b22d-8c13d339f17a,jonathan@wallaroo.ai,2025-08-May 16:00:09,2025-08-May 16:00:09
37,edge-pipeline,63548b0c-efa1-4127-9621-e941716eee9a,1549,edge-publish-demo,,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-main-6132,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-pipeline:63548b0c-efa1-4127-9621-e941716eee9a,younes.amar@wallaroo.ai,2025-12-May 00:49:05,2025-12-May 00:49:05
38,edge-pipeline,d6859c4b-cda5-4cd8-a95b-b531af4d4741,1549,edge-publish-demo,,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-main-6132,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-pipeline:d6859c4b-cda5-4cd8-a95b-b531af4d4741,younes.amar@wallaroo.ai,2025-12-May 00:49:15,2025-12-May 00:49:15
39,edge-pipeline,fd9e3a36-1c75-4982-bb17-7efbf6a63543,1549,edge-publish-demo,,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-main-6136,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-pipeline:fd9e3a36-1c75-4982-bb17-7efbf6a63543,john.hummel@wallaroo.ai,2025-13-May 13:35:50,2025-13-May 13:35:50
40,edge-pipeline,59924841-88f1-4f7b-8dc6-bc0b1d6ec65c,1549,edge-publish-demo,,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-main-6136,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-pipeline:59924841-88f1-4f7b-8dc6-bc0b1d6ec65c,john.hummel@wallaroo.ai,2025-13-May 13:36:06,2025-13-May 13:36:06


## DevOps - Pipeline Edge Deployment

Once a pipeline is deployed to the Edge Registry service, it can be deployed in environments such as Docker, Kubernetes, or similar container running services by a DevOps engineer as a Wallaroo Server.

The following guides will demonstrate publishing a Wallaroo Pipeline as a Wallaroo Server.

### Add Edge Location

Wallaroo Servers can optionally connect to the Wallaroo Ops instance and transmit their inference results.  These are added to the pipeline logs for the published pipeline the Wallaroo Server is associated with.

Wallaroo Servers are added with the `wallaroo.pipeline_publish.add_edge(name: string)` method.  The `name` is the unique primary key for each edge added to the pipeline publish and must be unique.

This returns a Publish Edge with the following fields:

| Field | Type | Description |
|---|---|---|
| `id` | Integer | The integer ID of the pipeline publish.|
| `created_at` | DateTime | The DateTime of the pipeline publish. |
| `docker_run_variables` | String | The Docker variables in UUID format that include the following: The `BUNDLE_VERSION`, `EDGE_NAME`, `JOIN_TOKEN_`, `OPSCENTER_HOST`, `PIPELINE_URL`, and `WORKSPACE_ID`.
| `engine_config` | String | The Wallaroo `wallaroo.deployment_config.DeploymentConfig` for the pipeline. |
| `pipeline_version_id` | Integer | The integer identifier of the pipeline version published. |
| `status` | String | The status of the publish.  `Published` is a successful publish.|
| `updated_at` | DateTime | The DateTime when the pipeline publish was updated. |
| `user_images` | List(String) | User images used in the pipeline publish. |
| `created_by` | String | The UUID of the Wallaroo user that created the pipeline publish. |
| `engine_url` | String | The URL for the published pipeline's Wallaroo engine in the OCI registry. |
| `error` | String | Any errors logged. |
| `helm` | String | The helm chart, helm reference and helm version. |
| `pipeline_url` | String | The URL for the published pipeline's container in the OCI registry. |
| `pipeline_version_name` | String | The UUID identifier of the pipeline version published. |
| `additional_properties` | String | Any other identities. |

In [50]:
xgb_edge = xgb_pub.add_edge("xgb-ccfraud-edge-publish")
display(xgb_edge)

keras_edge = keras_pub.add_edge("keras-ccfraud-publish")
display(keras_edge)


ID,1
Pipeline Name,edge-pipeline
Pipeline Version,6d0dba17-c135-4d6e-ba84-f80cd1497aec
Status,Published
Engine URL,ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.4.0-5879
Pipeline URL,ghcr.io/wallaroolabs/doc-samples/pipelines/edge-pipeline:6d0dba17-c135-4d6e-ba84-f80cd1497aec
Helm Chart URL,oci://ghcr.io/wallaroolabs/doc-samples/charts/edge-pipeline
Helm Chart Reference,ghcr.io/wallaroolabs/doc-samples/charts@sha256:0d7e8342fec6caec6cc06cefe37d7b2ed327ec18ec7095567b8904adc82b72cb
Helm Chart Version,0.0.1-6d0dba17-c135-4d6e-ba84-f80cd1497aec
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 1.0, 'memory': '900Mi'}, 'requests': {'cpu': 1.0, 'memory': '900Mi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}, 'engineAux': {'autoscale': {'type': 'none'}, 'images': {}}}"
User Images,[]


ID,2
Pipeline Name,edge-pipeline
Pipeline Version,a8873dfa-f42f-48b1-8577-9a3f60f96ec8
Status,Published
Engine URL,ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.4.0-5879
Pipeline URL,ghcr.io/wallaroolabs/doc-samples/pipelines/edge-pipeline:a8873dfa-f42f-48b1-8577-9a3f60f96ec8
Helm Chart URL,oci://ghcr.io/wallaroolabs/doc-samples/charts/edge-pipeline
Helm Chart Reference,ghcr.io/wallaroolabs/doc-samples/charts@sha256:b319f074739441dd0970a29a4168af315594f1febc96beaa8cb0d9f6e5dcf03e
Helm Chart Version,0.0.1-a8873dfa-f42f-48b1-8577-9a3f60f96ec8
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 1.0, 'memory': '900Mi'}, 'requests': {'cpu': 1.0, 'memory': '900Mi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}, 'engineAux': {'autoscale': {'type': 'none'}, 'images': {}}}"
User Images,[]


## DevOps Deployment

We now have our pipeline published to our Edge Registry service.  We can deploy this in a x86 environment running Docker that is logged into the same registry service that we deployed to.

For more details, check with the documentation on your artifact service.  The following are provided for the three major cloud services:

* [Set up authentication for Docker](https://cloud.google.com/artifact-registry/docs/docker/authentication)
* [Authenticate with an Azure container registry](https://learn.microsoft.com/en-us/azure/container-registry/container-registry-authentication?tabs=azure-cli)
* [Authenticating Amazon ECR Repositories for Docker CLI with Credential Helper](https://aws.amazon.com/blogs/compute/authenticating-amazon-ecr-repositories-for-docker-cli-with-credential-helper/)

Once a pipeline is deployed to the Edge Registry service, it can be deployed in environments such as Docker, Kubernetes, or similar container running services by a DevOps engineer.  For full details, see [How to Publish and Deploy AI Workloads for For Edge/Multicloud Model Deployments](https://docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-run-anywhere/wallaroo-model-operations-run-anywhere-deploy/).  The pipeline publishes `Docker Run Command` and `Helm Install Command` provide templates for deployment.

For our examples, we will deploy the XGB version of the pipeline to port `8080`, and the Keras version to port `8081`.

## Edge Deployed Pipeline API Endpoints

Once deployed, we can check the pipelines and models available.  We'll use a `curl` command, but any HTTP based request will work the same way.

The endpoint `/pipelines` returns:

* **id** (*String*):  The name of the pipeline.
* **status** (*String*):  The status as either `Running`, or `Error` if there are any issues.

For this example, the deployment is made on a machine called `localhost`.  Replace this URL with the URL of you edge deployment.


In [51]:
!curl testboy.local:8080/pipelines

{"pipelines":[{"id":"edge-pipeline","version":"6d0dba17-c135-4d6e-ba84-f80cd1497aec","status":"Running"}]}

In [52]:
!curl testboy.local:8081/pipelines

{"pipelines":[{"id":"edge-pipeline","version":"a8873dfa-f42f-48b1-8577-9a3f60f96ec8","status":"Running"}]}

The endpoint `/models` returns a List of models with the following fields:

* **name** (*String*): The model name.
* **sha** (*String*): The sha hash value of the ML model.
* **status** (*String*):  The status of either Running or Error if there are any issues.
* **version** (*String*):  The model version.  This matches the version designation used by Wallaroo to track model versions in UUID format.

In [53]:
!curl testboy.local:8080/models

{"models":[{"sha":"054810e3e3ebbdd34438d9c1a08ed6a6680ef10bf97b9223f78ebf38e14b3b52","name":"ccfraud-xgboost","version":"b847f43d-0795-4100-b75b-632e291628d5","status":"Running","model_version_id":6}]}

In [54]:
!curl testboy.local:8081/models

{"models":[{"sha":"bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507","name":"ccfraud-keras","version":"5afd7f34-412c-4e0c-a9a9-714d24d5e632","status":"Running","model_version_id":7}]}

### Edge Inference Endpoint

The inference endpoint takes the following pattern:

* `/infer`:  The inference endpoint remains the same regardless of the pipeline or models deployed.  This allows publish replacements without altering the inference endpoint used by other applications.

Wallaroo inference endpoint URLs accept the following data inputs through the `Content-Type` header:

* `Content-Type: application/vnd.apache.arrow.file`: For Apache Arrow tables.
* `Content-Type: application/json; format=pandas-records`: For pandas DataFrame in record format.

Once deployed, we can perform an inference through the deployment URL.

The endpoint returns `Content-Type: application/json; format=pandas-records` by default with the following fields:

* **check_failures** (*List[Integer]*): Whether any validation checks were triggered.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Management: Anomaly Testing](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/#anomaly-testing).
* **elapsed** (*List[Integer]*): A list of time in nanoseconds for:
  * [0] The time to serialize the input.
  * [1...n] How long each step took.
* **model_name** (*String*): The name of the model used.
* **model_version** (*String*): The version of the model in UUID format.
* **original_data**: The original input data.  Returns `null` if the input may be too long for a proper return.
* **outputs** (*List*): The outputs of the inference result separated by data type, where each data type includes:
  * **data**: The returned values.
  * **dim** (*List[Integer]*): The dimension shape returned.
  * **v** (*Integer*): The vector shape of the data.
* **pipeline_name**  (*String*): The name of the pipeline.
* **shadow_data**: Any shadow deployed data inferences in the same format as **outputs**.
* **time** (*Integer*): The time since UNIX epoch.

In [55]:
!curl -X POST testboy.local:8080/infer \
    -H "Content-Type: application/vnd.apache.arrow.file" \
    --data-binary @./data/cc_data_10k.arrow > xgb-edge-infer.df.json

df_xgb = pd.read_json("xgb-edge-infer.df.json", orient="records")
df_xgb.head(5)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8232k  100 7069k  100 1162k   457k  77114  0:00:15  0:00:15 --:--:--  460k


,time,in,out,anomaly,metadata
0,1736445397541,"{'tensor': [-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.7687824400000001, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204422999999999, -0.10414918000000001, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463211999999999]}",{'variable': [1.0094898]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-xgboost"",""model_sha"":""054810e3e3ebbdd34438d9c1a08ed6a6680ef10bf97b9223f78ebf38e14b3b52""}', 'pipeline_version': '6d0dba17-c135-4d6e-ba84-f80cd1497aec', 'elapsed': [1947085, 24555755], 'dropped': [], 'partition': 'xgb-ccfraud-edge-publish'}"
1,1736445397541,"{'tensor': [-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.7687824400000001, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204422999999999, -0.10414918000000001, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463211999999999]}",{'variable': [1.0094898]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-xgboost"",""model_sha"":""054810e3e3ebbdd34438d9c1a08ed6a6680ef10bf97b9223f78ebf38e14b3b52""}', 'pipeline_version': '6d0dba17-c135-4d6e-ba84-f80cd1497aec', 'elapsed': [1947085, 24555755], 'dropped': [], 'partition': 'xgb-ccfraud-edge-publish'}"
2,1736445397541,"{'tensor': [-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.7687824400000001, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204422999999999, -0.10414918000000001, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463211999999999]}",{'variable': [1.0094898]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-xgboost"",""model_sha"":""054810e3e3ebbdd34438d9c1a08ed6a6680ef10bf97b9223f78ebf38e14b3b52""}', 'pipeline_version': '6d0dba17-c135-4d6e-ba84-f80cd1497aec', 'elapsed': [1947085, 24555755], 'dropped': [], 'partition': 'xgb-ccfraud-edge-publish'}"
3,1736445397541,"{'tensor': [-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.7687824400000001, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204422999999999, -0.10414918000000001, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463211999999999]}",{'variable': [1.0094898]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-xgboost"",""model_sha"":""054810e3e3ebbdd34438d9c1a08ed6a6680ef10bf97b9223f78ebf38e14b3b52""}', 'pipeline_version': '6d0dba17-c135-4d6e-ba84-f80cd1497aec', 'elapsed': [1947085, 24555755], 'dropped': [], 'partition': 'xgb-ccfraud-edge-publish'}"
4,1736445397541,"{'tensor': [0.5817662, 0.09788155, 0.15468194, 0.4754102, -0.19788623, -0.45043448, 0.016654044, -0.025607055, 0.09205616, -0.27839172, 0.059329946, -0.019658541, -0.42250833, -0.12175389, 1.5473095, 0.23916228, 0.3553975, -0.76851654, -0.7000849, -0.11900433, -0.34505169999999996, -1.1065114, 0.25234112000000003, 0.020944182000000002, 0.21992674, 0.25406894, -0.04502251, 0.10867739, 0.25471792]}",{'variable': [-1.9073485999999998e-06]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-xgboost"",""model_sha"":""054810e3e3ebbdd34438d9c1a08ed6a6680ef10bf97b9223f78ebf38e14b3b52""}', 'pipeline_version': '6d0dba17-c135-4d6e-ba84-f80cd1497aec', 'elapsed': [1947085, 24555755], 'dropped': [], 'partition': 'xgb-ccfraud-edge-publish'}"


In [56]:
!curl -X POST testboy.local:8081/infer \
    -H "Content-Type: application/vnd.apache.arrow.file" \
    --data-binary @./data/cc_data_10k.arrow > xgb-edge-infer.df.json

df_keras = pd.read_json("xgb-edge-infer.df.json", orient="records")
df_keras.head(5)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8151k  100 6989k  100 1162k   412k  70298  0:00:16  0:00:16 --:--:--  360kk   377k   147k  0:00:18  0:00:07  0:00:11  462k11k  100 1162k   421k    97k  0:00:16  0:00:11  0:00:05  510kk  100 1162k   414k  74977  0:00:16  0:00:15  0:00:01  417k


,time,in,out,anomaly,metadata
0,1736445413514,"{'tensor': [-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.7687824400000001, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204422999999999, -0.10414918000000001, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463211999999999]}",{'dense_1': [0.99300325]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-keras"",""model_sha"":""bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507""}', 'pipeline_version': 'a8873dfa-f42f-48b1-8577-9a3f60f96ec8', 'elapsed': [1959561, 6941435], 'dropped': [], 'partition': 'keras-ccfraud-publish'}"
1,1736445413514,"{'tensor': [-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.7687824400000001, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204422999999999, -0.10414918000000001, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463211999999999]}",{'dense_1': [0.99300325]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-keras"",""model_sha"":""bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507""}', 'pipeline_version': 'a8873dfa-f42f-48b1-8577-9a3f60f96ec8', 'elapsed': [1959561, 6941435], 'dropped': [], 'partition': 'keras-ccfraud-publish'}"
2,1736445413514,"{'tensor': [-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.7687824400000001, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204422999999999, -0.10414918000000001, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463211999999999]}",{'dense_1': [0.99300325]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-keras"",""model_sha"":""bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507""}', 'pipeline_version': 'a8873dfa-f42f-48b1-8577-9a3f60f96ec8', 'elapsed': [1959561, 6941435], 'dropped': [], 'partition': 'keras-ccfraud-publish'}"
3,1736445413514,"{'tensor': [-1.0603298, 2.3544967, -3.5638788, 5.138735, -1.2308457, -0.7687824400000001, -3.5881228, 1.8880838, -3.2789674, -3.9563255, 4.099344, -5.653918, -0.8775733, -9.131571, -0.6093538, -3.7480276, -5.0309124, -0.8748149, 1.9870535, 0.7005486, 0.9204422999999999, -0.10414918000000001, 0.32295644, -0.74181414, 0.038412016, 1.0993439, 1.2603409, -0.14662448, -1.4463211999999999]}",{'dense_1': [0.99300325]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-keras"",""model_sha"":""bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507""}', 'pipeline_version': 'a8873dfa-f42f-48b1-8577-9a3f60f96ec8', 'elapsed': [1959561, 6941435], 'dropped': [], 'partition': 'keras-ccfraud-publish'}"
4,1736445413514,"{'tensor': [0.5817662, 0.09788155, 0.15468194, 0.4754102, -0.19788623, -0.45043448, 0.016654044, -0.025607055, 0.09205616, -0.27839172, 0.059329946, -0.019658541, -0.42250833, -0.12175389, 1.5473095, 0.23916228, 0.3553975, -0.76851654, -0.7000849, -0.11900433, -0.34505169999999996, -1.1065114, 0.25234112000000003, 0.020944182000000002, 0.21992674, 0.25406894, -0.04502251, 0.10867739, 0.25471792]}",{'dense_1': [0.0010916889]},{'count': 0},"{'last_model': '{""model_name"":""ccfraud-keras"",""model_sha"":""bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507""}', 'pipeline_version': 'a8873dfa-f42f-48b1-8577-9a3f60f96ec8', 'elapsed': [1959561, 6941435], 'dropped': [], 'partition': 'keras-ccfraud-publish'}"
